In [ ]:
# Go to Runtime -> Change runtime type -> Set hardware accelerator to a GPU

In [ ]:

!git clone --recursive https://github.com/szymanowiczs/splatter-image.git
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git

Cloning into 'splatter-image'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 183 (delta 68), reused 47 (delta 47), pack-reused 79 (from 1)
Receiving objects: 100% (183/183), 2.96 MiB | 7.55 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 1053, done.
remote: Total 1053 (delta 0), reused 0 (delta 0), pack-reused 1053 (from 1)
Receiving objects: 100% (1053/1053), 78.70 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (604/604), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'su

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/splatter-image
!pip install -r requirements.txt

/content/splatter-image
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of fastapi to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of fastapi to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer

In [ ]:
# Get correct package versions
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

%cd /content/gaussian-splatting
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.0;7.5;8.0+PTX"
!pip install -e submodules/diff-gaussian-rasterization
# !pip install gsplat

/content/gaussian-splatting
Obtaining file:///content/gaussian-splatting/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Running setup.py develop for diff_gaussian_rasterization


In [ ]:
from google.colab import drive
import torch
import numpy as np
import matplotlib.pyplot as plt

#drive.mount('/content/drive')
print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.8.0+cu126
CUDA Version: 12.6


In [ ]:
# download srn cars dataset

%cd /content
!mkdir SRN
%cd /content/SRN
!mkdir srn_cars
%cd /content/SRN/srn_cars
!gdown --id 19yDsEJjx9zNpOKz9o6AaK-E8ED6taJWU -O cars.zip
!unzip cars.zip

Streaming output truncated to the last 5000 lines.
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000228.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000032.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000015.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000092.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000121.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000192.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000200.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000074.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000142.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000166.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000018.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000077.txt  
  inflating: cars_val/87ee241d3d0d1dda4ff3c6764341833/pose/000033.txt  
  inflating: 

In [ ]:
# before running eval: in line 19 of content/splatter-image/eval.py
# there is the line from datasets.dataset_factory but that clashes
# with datasets from huggingface. Change the folder name of splatter-image/datasets
# to splatter-image/splatter_image_datasets (or whatever you want)
# and replace line 19 accordingly.

# Also in srn.py line 14, SHAPENET_DATASET_ROOT = "/content/SRN"
# replace similar lines in co3d.py, gso.py, nmr.py and objaverse.py with temporary names
# just to not trigger the assert not None

# in eval.py line 273, change to ckpt_loaded = torch.load(model_path, map_location=device, weights_only=False)

# in splatter-image/gaussian_renderer/__init__.py, add antialiasing=False (or True, i don't know)
# to raster_settings in line 41
# replace line 85 with
# temp = rasterizer(
#         means3D = means3D,
#         means2D = means2D,
#         shs = shs,
#         colors_precomp = colors_precomp,
#         opacities = opacity,
#         scales = scales,
#         rotations = rotations,
#         cov3D_precomp = cov3D_precomp)
#     rendered_image = temp[0]
#     radii = temp[1]

%cd /content/splatter-image
!python eval.py cars

/content/splatter-image
Evaluating on dataset cars
Will load a model released with the paper.
Not saving any renders (only computing scores). To save renders use flag --save_vis
Loaded model!
704
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth
 27% 192/704 [09:35<25:49,  3